# ***FAQ CHATBOT***

---


An FAQ chatbot can respond to user queries based on a predefined set of frequently asked questions  using Natural Language Processing (NLP). It understands different phrasings of the same question by using machine learning models like sentence embeddings (BERT) instead of simple keyword matching. This allows it to provide accurate, instant, and consistent responses while also supporting multi-turn conversations and entity recognition (NER). Advanced chatbots can also log interactions for analysis, improving efficiency and reducing human workload.

# **Importing Required Libraries**

In [1]:
import spacy
import json
from sentence_transformers import SentenceTransformer, util

# **Loading Predefined FAQs**

In [10]:
def load_faq():
    return {
        "Hi": "Hello! How can I assist you today?",
        "hello":"hello! How can I assist you today?",
        "What is your name?": "My name is FAQ Chatbot.",
        "I need help": "Sure, I'm here to help. What do you need assistance with?",
        "Thanks":"Your welcome",
        "What are your working hours?": "We are open from 9 AM to 6 PM, Monday to Friday.",
        "How can I reset my password?": "You can reset your password by clicking on 'Forgot Password' on the login page.",
        "Where is your office located?": "Our office is located at Thampanoor ,Trivandrum.",
        "What is your refund policy?": "You can request a refund within 30 days of purchase.",
        "How can I contact customer support?": "You can contact our support team via email at indship@gmail.com or call us at (123) 456-7890.",
        "Do you offer international shipping?": "Yes, we offer international shipping. Delivery times vary based on location.",
        "How do I track my order?": "You can track your order by entering your tracking number on our tracking page.",
        "Can I cancel my order?": "You can cancel your order within 24 hours of placing it by contacting our support team.",
        "What payment methods do you accept?": "We accept credit/debit cards, PayPal, and bank transfers.",
        "How do I update my account information?": "You can update your account details by logging into your profile settings page.",
        "Do you have a mobile app?": "Yes, our mobile app is available on both Android and iOS platforms.",
        "What services do you offer?": "We offer a variety of services, including technical support, consultations, and training sessions.",
        "How do I create an account?": "You can create an account by clicking 'Sign Up' on our homepage and following the registration steps.",
        "Is my personal information secure?": "Yes, we use encryption and security measures to protect your personal information.",
        "Do you provide discounts for students?": "Yes, we offer special discounts for students. Contact our support team for more details.",
        "Can I change my subscription plan?": "Yes, you can upgrade or downgrade your subscription plan from your account settings.",
        "How do I delete my account?": "To delete your account, go to settings and select 'Delete Account,' or contact support for assistance."
    }

# **Saving Conversation Logs**

In [3]:

def save_log(conversations):
    with open("chat_log.json", "w") as f:
        json.dump(conversations, f, indent=4)

# **The Main Chatbot Function**

The chatbot() function initializes the spaCy NLP model for Named Entity Recognition(NER) ,And a pretrained BERT-based model (SentenceTransformer) for computing the sentence embeddings. It loads the FAQ dataset, encodes all FAQ questions into the embeddings,and enters to a loop to interact with the user . For each user query, it computes semantic similarity between query and stored FAQ embeddings using cosine similarity. If the best match exceeds a confidence threshold, it returns the corresponding FAQ answer; otherwise, it asks the user to rephrase. Additionally, it extracts named entities from the query and logs the conversation in a JSON file for future analysis.The chatbot continues to run until the user tyoe "Exit".

In [4]:
def chatbot():
    nlp = spacy.load("en_core_web_sm")
    model = SentenceTransformer("all-MiniLM-L6-v2")
    faq = load_faq()
    questions, answers = list(faq.keys()), list(faq.values())
    question_embeddings = model.encode(questions, convert_to_tensor=True)
    conversations = []

    print("FAQ Chatbot: Type 'exit' to stop.")
    while True:
        user_query = input("You: ").strip()
        if user_query.lower() == 'exit':
            print("Chatbot: Goodbye!")
            save_log(conversations)
            break

        query_embedding = model.encode(user_query, convert_to_tensor=True)
        similarity_scores = util.pytorch_cos_sim(query_embedding, question_embeddings)[0]
        best_match_idx = similarity_scores.argmax().item()
        confidence = similarity_scores[best_match_idx].item()

        if confidence > 0.5:
            response = answers[best_match_idx]
        else:
            response = "I'm not sure. Could you rephrase your question?"

        print(f"Chatbot: {response}")
        conversations.append({"User": user_query, "Chatbot": response})

        # Named Entity Recognition (NER)
        entities = [(ent.text, ent.label_) for ent in nlp(user_query).ents]
        if entities:
            print(f"(Recognized Entities: {entities})")


#  Running the Chatbot

This ensures that the chatbot runs only when the script is executed, not when imported as a module .


In [11]:
if __name__ == "__main__":
    chatbot()


FAQ Chatbot: Type 'exit' to stop.
You: hello
Chatbot: hello! How can I assist you today?
You: whats your name
Chatbot: My name is FAQ Chatbot.
You: i need help
Chatbot: Sure, I'm here to help. What do you need assistance with?
You: i forgot my password
Chatbot: You can reset your password by clicking on 'Forgot Password' on the login page.
You: i need customer care number
Chatbot: You can contact our support team via email at indship@gmail.com or call us at (123) 456-7890.
You: thanks
Chatbot: Your welcome
You: "Can I cancel my order?
Chatbot: You can cancel your order within 24 hours of placing it by contacting our support team.
You: Do you provide discounts for students?
Chatbot: Yes, we offer special discounts for students. Contact our support team for more details.
You: exit
Chatbot: Goodbye!
